# Обучение моделей

## Импорт библиотек

In [1]:
import os, sys

PROJECT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
print(PROJECT_DIR)
sys.path.append(PROJECT_DIR)

/workspaces/street_tree


In [2]:
import pandas as pd 
from src.config import PATH_RAW, PATH_PROCESSED, URL, NAME_DATASET, PATH_MODELS, output_path
from src.dowload_datafraime import download_file
from src.visualization import plot_corr_matrix
from src.preprocessing import (df_fillna, split_problems, convert_to_bool, encode_and_save_categorical, 
                               load_and_encode_categorical, split_and_save)


2025-03-23 12:16:29,790 - src.utils - INFO - Logger is ready
2025-03-23 12:16:29,864 - src.utils - INFO - This is a test log message.
2025-03-23 12:16:29,865 - src.utils - INFO - PROJECT_DIR: /workspaces/street_tree
2025-03-23 12:16:29,866 - src.utils - INFO - PATH_LOGS: /workspaces/street_tree/logs
2025-03-23 12:16:29,867 - src.utils - INFO - PATH_DATA: /workspaces/street_tree/data
2025-03-23 12:16:29,867 - src.utils - INFO - PATH_RAW: /workspaces/street_tree/data/raw
2025-03-23 12:16:29,868 - src.utils - INFO - PATH_PROCESSED: /workspaces/street_tree/data/processed
2025-03-23 12:16:29,869 - src.utils - INFO - PATH_MODELS: /workspaces/street_tree/models
2025-03-23 12:16:29,870 - src.utils - INFO - PATH_REPORTS: /workspaces/street_tree/reports
2025-03-23 12:16:29,870 - src.utils - INFO - PATH_DOCS: /workspaces/street_tree/docs
2025-03-23 12:16:29,871 - src.utils - INFO - PATH_SRC: /workspaces/street_tree/src
2025-03-23 12:16:29,871 - src.utils - INFO - output_path: /workspaces/street_t

## Предобработка

In [ ]:
#download_file(URL,PATH_RAW, output_path)

: 

In [ ]:
df = pd.read_csv(f'{PATH_RAW}/{NAME_DATASET}.csv')
df = df.dropna(subset=['health'])

X_train, X_test, y_train, y_test = split_and_save(df.drop('health', axis=1), df.health, output_dir=PATH_PROCESSED, size=0.1, name_train='train.csv', name_test='test.csv')

df.shape

In [ ]:
X_train.columns = [col.lower().replace(' ', '_') for col in X_train.columns]
df_drop_columns = ['status', 'state', 'tree_id', 'created_at', 'stump_diam', 'address', 'spc_common',
                   'borocode', 'x_sp', 'y_sp', 'council_district', 'census_tract', 'nta_name',
                   'bin', 'zip_city', 'community_board', 'bbl']
X_train = X_train.drop(columns=df_drop_columns)
X_train.shape


(586954, 27)

In [ ]:
X_train = df_fillna(X_train)
X_train.shape

/workspaces/street_tree/src/preprocessing.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['spc_latin'].fillna('No observation',inplace=True)
/workspaces/street_tree/src/preprocessing.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].metho

(586954, 27)

In [ ]:
X_train = split_problems(X_train, created_columns=False)
X_train.shape

2025-03-23 12:02:45,329 - src.utils - INFO - {'sneakers', 'trunkother', 'branchlights', 'metalgrates', 'branchother', 'trunklights', 'stones', 'rootother', 'wiresrope', 'noproblem'} - уникальные проблемы


(586954, 27)

In [ ]:
X_train = convert_to_bool(X_train)
X_train.shape

2025-03-23 12:02:47,713 - src.utils - INFO - Значения преобразованы в булевые


(586954, 27)

In [ ]:
categorical_columns = X_train.select_dtypes(include='object').columns.tolist()
categorical_columns

['spc_latin', 'steward', 'guards', 'user_type', 'borough', 'nta']

In [ ]:
X_train, label_encoders = encode_and_save_categorical(X_train, categorical_columns, PATH_MODELS, 'label_encoders.pkl')
X_train.shape

(586954, 27)

In [ ]:
# Вывод значений, закодированных энкодером для столбца 'health', вместе с соответствующими им числовыми кодами
health_classes = label_encoders['spc_latin'].classes_
health_codes = label_encoders['spc_latin'].transform(health_classes)

# Создание DataFrame для удобного отображения
health_mapping = pd.DataFrame({'spc_latin': health_classes, 'Code': health_codes})
print(health_mapping)

             spc_latin  Code
0                 Acer     0
1    Acer buergerianum     1
2       Acer campestre     2
3         Acer ginnala     3
4         Acer griseum     4
..                 ...   ...
128   Tsuga canadensis   128
129    Ulmus americana   129
130   Ulmus parvifolia   130
131       Ulmus pumila   131
132    Zelkova serrata   132

[133 rows x 2 columns]


## Обучим модели

In [ ]:
# Разделим X_train на обучающую и валидационную выборки и y_train на соответствующие им значения
X_train, X_val, y_train, y_val = split_and_save(X_train, y_train, output_dir=PATH_PROCESSED, size=0.2, name_train='train.csv', name_test='valid.csv')

2025-03-23 12:02:53,499 - src.utils - INFO - Data successfully saved to: /workspaces/street_tree/data/processed


In [ ]:
y_train.unique()

array(['Poor', 'Good', 'Fair'], dtype=object)

In [ ]:
import joblib

# Define the mapping
target_mapping = {'Poor': 0, 'Fair': 1, 'Good': 2}

y_train = y_train.map(target_mapping)
y_val = y_val.map(target_mapping)

# Save the mapping to a file
joblib.dump(target_mapping, f'{PATH_MODELS}/label_encoders_target.pkl')

print("Target label encoder saved to label_encoders_target.pkl")

Target label encoder saved to label_encoders_target.pkl


In [ ]:
from sklearn.preprocessing import StandardScaler
import joblib

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
# Save the scaler
joblib.dump(scaler, f'{PATH_MODELS}/scaler.pkl')
print("Scaler saved to scaler.pkl")

Scaler saved to scaler.pkl


In [ ]:
X_train.shape

(469563, 27)

In [ ]:
%load_ext autoreload
%autoreload 2

from src.modeling import TabularNN

# Instantiate the model
model = TabularNN(X_train, y_train, X_val, y_val, hidden_dims=[128, 64], model_path=f'{PATH_MODELS}/tabular_model.pth')

# Train the model
model.train_model(epochs=2, learning_rate=0.001)



2025-03-23 12:02:59,237 - src.utils - INFO - Unique classes during training: [0 1 2]
2025-03-23 12:02:59,238 - src.utils - INFO - Output dimension during training: 3
2025-03-23 12:03:17,187 - src.utils - INFO - Epoch 1/2 - Train Loss: 7709.2499, Val Loss: 1898.4697, LR: 0.001000
2025-03-23 12:03:17,187 - src.utils - INFO - Accuracy: 0.5982, F1-Score: 0.6523, AUC-ROC: 0.6477
2025-03-23 12:03:17,188 - src.utils - INFO - Confusion Matrix:
2070	1002	1755
4982	4367	8022
14804	16600	63789
2025-03-23 12:03:17,196 - src.utils - INFO - Best model saved based on highest AUC-ROC.
2025-03-23 12:03:31,146 - src.utils - INFO - Epoch 2/2 - Train Loss: 7623.7223, Val Loss: 1889.5846, LR: 0.001000
2025-03-23 12:03:31,147 - src.utils - INFO - Accuracy: 0.5951, F1-Score: 0.6522, AUC-ROC: 0.6679
2025-03-23 12:03:31,147 - src.utils - INFO - Confusion Matrix:
2072	1190	1565
4865	4784	7722
16198	15994	63001
2025-03-23 12:03:31,150 - src.utils - INFO - Best model saved based on highest AUC-ROC.


In [ ]:
import pandas as pd
import numpy as np
from src.modeling import TabularNN
from src.config import PATH_MODELS  # Import PATH_MODELS
import torch

# Define the path to the saved model
MODEL_PATH = f"{PATH_MODELS}/tabular_model.pth"

# Load the checkpoint to get the parameters
checkpoint = torch.load(MODEL_PATH)
input_dim = checkpoint['input_dim']
hidden_dims = checkpoint['hidden_dims']
output_dim = checkpoint['output_dim']
target_mapping = checkpoint['target_mapping']

# Dummy data for loading the model (same shape as training data)
# Replace 27 with your actual input dimension
dummy_X = pd.DataFrame(np.zeros((1, input_dim)))  # Use a single row
# The dummy y MUST contain all possible class labels
dummy_y = pd.Series([0, 1, 2])  # All possible class labels

# Load the model
# Pass the dummy data with the correct input dimension
loaded_model = TabularNN(dummy_X, dummy_y, dummy_X, dummy_y, hidden_dims=hidden_dims)
loaded_model.load_state_dict(checkpoint['model_state_dict'])
loaded_model.eval()

print("Model loaded successfully!")

loaded_model
# Now you can use the loaded_model for predictions
# Example:
# predictions = loaded_model.predict(new_data)

2025-03-23 12:05:47,216 - src.utils - INFO - Unique classes during training: [0 1 2]
2025-03-23 12:05:47,217 - src.utils - INFO - Output dimension during training: 3


AssertionError: Size mismatch between tensors

In [ ]:
dummy_X

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
%load_ext autoreload
%autoreload 2
from src.modeling_catboost import CatBoostModelTrain

catboost_model = CatBoostModelTrain(model_dir=PATH_MODELS)
catboost_model.train(X_train, y_train, X_val, y_val)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0:	learn: 1.0665870	test: 1.0666001	best: 1.0666001 (0)	total: 500ms	remaining: 8m 19s
100:	learn: 0.5502277	test: 0.5516981	best: 0.5516981 (100)	total: 34.2s	remaining: 5m 4s
200:	learn: 0.5351898	test: 0.5379727	best: 0.5379727 (200)	total: 1m 12s	remaining: 4m 49s
300:	learn: 0.5281540	test: 0.5319916	best: 0.5319916 (300)	total: 1m 47s	remaining: 4m 10s


KeyboardInterrupt: 